In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

In [5]:
import sys
sys.path.insert(1,'../covid-website')

from data import Data
from SIR import *

In [6]:
params = {
    "population": 100e3,
    "i0": 1,
    "p0": 1,
    "f0": 0,
    "gamma_infec": 1/6,   #1/length of infectious period in days
    "beta0": 2.5 * (1/6),   #R0 =beta/gamma
    "death_rate": 0.5e-2,
    "detection_rate": 10e-2,
    "testing_segments":0,
    "mixing":1,
    "interv":"none"
}

n = 100
x = np.arange(n)
y = SIRF(x,params)

p = figure(plot_width=800, plot_height=600 )#, y_axis_type="log")
p.line(x, y[:,cI], line_width=3, line_color='black', line_dash='solid', alpha=0.3)

show(p)



In [7]:
source = 'Johns Hopkins'    
region = 'Europe'
state = 'Ireland'

cutoff_positive = 1
cutoff_death = 1
truncate = 0
window = 14

d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 
ed = d.excess_deaths()
h = d.hospitalization()

#if ed is not None:   
#    fig, axs = plt.subplots(1,1,figsize=(9,6))
#    axs.plot(d.xd[d.minD+1:], d.dfatalities,'+:', label='daily fatalities, COVID (JHU)')
#    axs.plot(ed['date'], ed['total_deaths']/7, "m:", label='daily fatalities, all causes (Economist)')
#    axs.plot(ed['date'], ed['excess_deaths']/7, "g:", label='excess deaths compared to historical baseline')
#    format_plot(axs,'linear','{}{} reported Covid fatalities compared to excess deaths vs.historical baseline'.format(region,state))

#if h is not None:
#    axs[0][0].plot(h['date'], h['hospitalizedCurrently'],'1:', label='in hospital')


In [8]:
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend

p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 )#, y_axis_type="log")

r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

legend = [
    ("COVID fatalities"   , [r0, r1]),
    ("COVID 7-day average"   , [r2]),
]

if ed is not None:
    r3 = p.line(ed['date'], ed['total_deaths']/7, line_width=1, line_color='black')
    r4 = p.line(ed['date'], ed['excess_deaths']/7, line_width=1, line_color='grey')

    legend.append(("Total Deaths", [r3]))
    legend.append(("Excess Deaths", [r4]))
    
p.add_layout(Legend(items=legend, location='center'), 'right')
   
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker

p.xaxis.major_label_orientation = math.pi/4

p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

#p.toolbar_location = None
show(p)